# Walmart Store Sales Forecasting Project

## Introduction

In this project, we will analyze and forecast sales data for Walmart stores using time series forecasting and regression techniques. Accurate sales prediction is crucial for Walmart, as it allows the company to manage inventory efficiently, plan promotional markdowns, and meet customer demand effectively. Unexpected fluctuations in demand can lead to stockouts or excessive inventory, impacting customer satisfaction and operational costs.

The dataset used in this project contains historical weekly sales data from 45 Walmart stores across various regions in the United States. Additionally, the dataset includes information on holiday events, economic indicators (such as CPI and the Unemployment Index), and promotional markdown events. These features make this dataset ideal for understanding both temporal and economic factors that impact retail sales.

## About the Dataset

The dataset provides:
- **Sales Data**: Weekly sales figures for each Walmart store.
- **Holiday Events**: Major U.S. holidays, such as the Super Bowl, Labour Day, Thanksgiving, and Christmas, which are expected to impact sales patterns.
- **Economic Indicators**: Key metrics, such as the Consumer Price Index (CPI) and Unemployment Index, which may influence consumer spending behavior.
- **Promotional Events**: Information on Walmart markdowns leading up to holidays, which can affect weekly sales.

This data presents an opportunity to explore how seasonal trends, holidays, economic conditions, and promotions affect sales. This is especially relevant for understanding demand patterns and making data-driven business decisions.

## Project Objectives

1. **Data Exploration**:
   - Analyze the dataset to understand its structure, missing values, and basic statistics.
   - Visualize patterns and trends to gain insights into factors influencing Walmart sales.

2. **Time Series Forecasting**:
   - Develop time series models to predict weekly sales over time.
   - Evaluate and fine-tune the model for improved forecasting accuracy.

3. **Anomaly Detection**:
   - Identify and analyze any anomalies or outliers in sales data, especially around major holidays and promotions.

4. **Regression Modeling**:
   - Build and evaluate regression models to predict sales based on features such as economic indicators, holiday events, and promotions.

5. **Model Evaluation**:
   - Compare model performances using metrics like R² and RMSE, and document findings for each model.

This notebook will guide you through each step, from data exploration to model development and evaluation. The goal is to achieve a comprehensive analysis and prediction model for Walmart store sales data, providing insights that could help Walmart optimize stock levels, plan promotions, and respond effectively to changes in demand.


In [7]:
# Standard Libraries
import pandas as pd
import numpy as np

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns

import os
import json
import joblib

In [8]:
os.chdir("C:/Users/USER/Desktop/GitHub/time-series-forecasting-sales-data")

# Data Loading and Initial Exploration

In [12]:
data = pd.read_csv('data/data.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    421570 non-null  int64  
 1   Store         421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   IsHoliday     421570 non-null  int64  
 4   Dept          421570 non-null  float64
 5   Weekly_Sales  421570 non-null  float64
 6   Temperature   421570 non-null  float64
 7   Fuel_Price    421570 non-null  float64
 8   MarkDown1     421570 non-null  float64
 9   MarkDown2     421570 non-null  float64
 10  MarkDown3     421570 non-null  float64
 11  MarkDown4     421570 non-null  float64
 12  MarkDown5     421570 non-null  float64
 13  CPI           421570 non-null  float64
 14  Unemployment  421570 non-null  float64
 15  Type          421570 non-null  int64  
 16  Size          421570 non-null  int64  
dtypes: float64(11), int64(5), object(1)
memory usage

,Unnamed: 0,Store,Date,IsHoliday,Dept,Weekly_Sales,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,0,1,2010-02-05,0,1.0,24924.50,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3,151315
1,1,1,2010-02-05,0,26.0,11737.12,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3,151315
2,2,1,2010-02-05,0,17.0,13223.76,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3,151315
3,3,1,2010-02-05,0,45.0,37.44,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3,151315
4,4,1,2010-02-05,0,28.0,1085.29,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,3,151315


In [11]:
data.shape

(421570, 17)